<a href="https://colab.research.google.com/github/miaomiao612/ChatGLM-6B/blob/main/FinetuneChatGLM6B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/miaomiao612/ChatGLM-6B.git

In [ ]:
!pip install -r /content/ChatGLM-6B/requirements.txt

In [ ]:
!pip install rouge_chinese nltk jieba datasets

In [4]:
import torch
import os

In [5]:
%cd /content/ChatGLM-6B/ptuning/

/content/ChatGLM-6B/ptuning


In [ ]:
!bash train.sh

In [7]:
from transformers import AutoConfig, AutoModel, AutoTokenizer

# 载入Tokenizer
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [9]:
config = AutoConfig.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True, pre_seq_len=128)
model = AutoModel.from_pretrained("THUDM/chatglm-6b", config=config, trust_remote_code=True)
prefix_state_dict = torch.load(os.path.join("/content/ChatGLM-6B/ptuning/output/adgen-chatglm-6b-pt-128-2e-2/checkpoint-1000", "pytorch_model.bin"))
new_prefix_state_dict = {}
for k, v in prefix_state_dict.items():
    if k.startswith("transformer.prefix_encoder."):
        new_prefix_state_dict[k[len("transformer.prefix_encoder."):]] = v
model.transformer.prefix_encoder.load_state_dict(new_prefix_state_dict)


Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Some weights of ChatGLMForConditionalGeneration were not initialized from the model checkpoint at THUDM/chatglm-6b and are newly initialized: ['transformer.prefix_encoder.embedding.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [10]:
# Comment out the following line if you don't use quantization
# model = model.quantize(4)
model = model.half().cuda()
model.transformer.prefix_encoder.float()
model = model.eval()

response, history = model.chat(tokenizer, "This is a street scene with a house on the left side of the image and a telephone pole on the right side of the image. There is a car parked on the street in front of the house. The house has a white facade with green shutters and a red roof. There are trees on either side of the street.", history=[])
response

'Change the scene to daytime'

In [11]:
# 与微调前进行对比
model_before = AutoModel.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True).half().cuda()
model_before = model_before.eval()

response_be, history = model_before.chat(tokenizer, "This is a street scene with a house on the left side of the image and a telephone pole on the right side of the image. There is a car parked on the street in front of the house. The house has a white facade with green shutters and a red roof. There are trees on either side of the street.", history=[])
response_be

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

'It sounds like you have provided a description of a street scene. Is there anything else you would like to know or add about the scene?'

In [18]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!git config --global user.email 'dddoctorr612@gmail.com'
!git config --global user.name 'Yifei SUN'


In [19]:
!git add .
!git commit -m "add checkpoint"


On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean


In [20]:
!git push

fatal: could not read Username for 'https://github.com': No such device or address
